# Supervised Fine-Tuning (SFT) Tutorial

This notebook demonstrates how to perform Supervised Fine-Tuning on language models to improve their instruction-following capabilities.

## What is SFT?

Supervised Fine-Tuning (SFT) is a post-training technique where we fine-tune a pre-trained language model on a curated dataset of high-quality examples. This helps the model learn to follow instructions better and produce more helpful, relevant responses.

## Key Components:
1. **Base Model**: A pre-trained language model
2. **Training Dataset**: High-quality instruction-response pairs
3. **Training Process**: Supervised learning with next-token prediction
4. **Evaluation**: Comparison of before/after model performance

---
*Based on Lesson 3 from DeepLearning.AI's "Post-training LLMs" course*

## Setup and Imports

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

import sys
import os

# Add the src directory to the path so we can import our modules
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

from utils.model_utils import load_model_and_tokenizer, test_model_with_questions, display_dataset
from training.sft_trainer import SFTTrainingPipeline
from datasets import load_dataset
import torch

## Configuration

In [ ]:
# Configuration
USE_GPU = False  # Set to True if you have a GPU available
MAX_SAMPLES = 100  # Reduce for faster training on limited resources

# Model and dataset configuration
BASE_MODEL = "HuggingFaceTB/SmolLM2-135M"  # Small model for demonstration
SFT_DATASET = "banghua/DL-SFT-Dataset"  # Curated SFT dataset

# Test questions for evaluation
test_questions = [
    "Give me a 1-sentence introduction of LLM.",
    "Calculate 1+1-1",
    "What's the difference between thread and process?",
    "Explain machine learning in simple terms.",
    "What are the applications of neural networks?"
]

print(f"Configuration:")
print(f"- Base model: {BASE_MODEL}")
print(f"- Dataset: {SFT_DATASET}")
print(f"- Max samples: {MAX_SAMPLES}")
print(f"- Use GPU: {USE_GPU}")

## Step 1: Load and Test Base Model

First, let's load the base model and see how it performs on our test questions before any fine-tuning.

In [ ]:
print("Loading base model...")
model, tokenizer = load_model_and_tokenizer(BASE_MODEL, USE_GPU)

print(f"\nModel loaded: {BASE_MODEL}")
print(f"Model device: {next(model.parameters()).device}")
print(f"Number of parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Test the base model
test_model_with_questions(
    model, tokenizer, test_questions,
    title="Base Model Performance (Before SFT)"
)

In [ ]:
# Clean up base model to free memory
del model, tokenizer
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("Base model cleaned up from memory.")

## Step 2: Load and Explore Training Dataset

Now let's load the SFT training dataset and examine its structure.

In [ ]:
# Load training dataset
print(f"Loading dataset: {SFT_DATASET}")
train_dataset = load_dataset(SFT_DATASET)["train"]

# Limit samples for demonstration
if MAX_SAMPLES and MAX_SAMPLES < len(train_dataset):
    train_dataset = train_dataset.select(range(MAX_SAMPLES))

print(f"Dataset size: {len(train_dataset)}")
print(f"Dataset columns: {train_dataset.column_names}")

In [ ]:
# Display sample training examples
print("Sample training examples:")
display_dataset(train_dataset, num_examples=3)

## Step 3: Set Up and Run SFT Training

Now we'll use our SFT training pipeline to fine-tune the model.

In [ ]:
# Initialize SFT training pipeline
print("Initializing SFT training pipeline...")
sft_pipeline = SFTTrainingPipeline(BASE_MODEL, use_gpu=USE_GPU)

# Setup training configuration
sft_pipeline.setup_training(
    train_dataset,
    learning_rate=8e-5,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    gradient_checkpointing=False,
    logging_steps=10
)

print("Training configuration set up successfully!")

In [ ]:
# Run training
print("Starting SFT training...")
print("This may take several minutes depending on your hardware and dataset size.")
print("-" * 50)

sft_pipeline.train()

print("-" * 50)
print("SFT training completed!")

## Step 4: Evaluate the Fine-Tuned Model

Let's test the fine-tuned model on the same questions and compare the results.

In [ ]:
# Evaluate the trained model
sft_pipeline.evaluate_model(
    test_questions,
    title="Fine-Tuned Model Performance (After SFT)"
)

## Step 5: Save the Fine-Tuned Model

Save the trained model for future use.

In [ ]:
# Save the trained model
output_dir = "../models/sft_trained_model"
sft_pipeline.save_model(output_dir)

print(f"Model saved to: {output_dir}")
print("You can now load this model for inference or further training.")

## Summary and Key Takeaways

### What we accomplished:

1. **Loaded a base model** and evaluated its performance
2. **Prepared a training dataset** with instruction-response pairs
3. **Fine-tuned the model** using Supervised Fine-Tuning
4. **Evaluated improvements** in model responses
5. **Saved the trained model** for future use

### Key observations:

- SFT helps models follow instructions better
- The quality of training data is crucial
- Even small models can benefit significantly from SFT
- Training time varies based on dataset size and hardware

### Next steps:

- Try SFT with larger models and datasets
- Experiment with different learning rates and training epochs
- Combine SFT with other post-training techniques like DPO
- Evaluate on domain-specific tasks

---
*This tutorial is based on the DeepLearning.AI "Post-training LLMs" course, Lesson 3.*